Adaptado de ["Classificação de Imagens"](https://www.tensorflow.org/tutorials/images/classification?hl=pt-br) dos [tutoriais do Tensorflow](https://www.tensorflow.org/tutorials/).

# Classificação de Imagens

<a href="https://colab.research.google.com/github/fabiobento/edge-ml/blob/main/computer-vision/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este tutorial mostra como classificar imagens de flores usando um modelo `tf.keras.Sequential` e carregar dados usando `tf.keras.utils.image_dataset_from_directory`. Ele demonstra os seguintes conceitos:


* Carregamento eficiente de um conjunto de dados do disco.
* Identificação de sobreajuste e aplicação de técnicas para atenuá-lo, incluindo aumento e eliminação de dados.

Este tutorial segue um fluxo de trabalho básico de aprendizado de máquina:

1. Examinar e entender os dados
2. Criar um pipeline de entrada
3. Criar o modelo
4. Treinar o modelo
5. Testar o modelo
6. Aprimore o modelo e repita o processo

## Configuração

Importe o TensorFlow e outras bibliotecas necessárias:

In [41]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

## Faça o download e explore o conjunto de dados

Este tutorial usa um conjunto de dados com cerca de 3.700 fotos de flores. O conjunto de dados contém cinco subdiretórios, um por classe:

```
flower_photo/
  daisy/
  dandelion/
  roses/
  sunflowers/
  tulips/
```

In [ ]:
import requests
import tarfile
import os
import pathlib

# Especificar aonde está armazenado o arquivo com as fotos compactadas
url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
file_name = "flower_photos.tgz"

# Baixar o arquivo compactado com as imagens do conjuntos de dados
response = requests.get(url, stream=True)
with open(file_name, "wb") as file:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            file.write(chunk)

print(f"Download concluído: {file_name}")

# 2. Extrair o arquivo .tgz
extract_path = os.getcwd()+'/datasets'
with tarfile.open(file_name, "r:gz") as tar:
    tar.extractall(path=extract_path)

data_dir = pathlib.Path(extract_path+'/'+file_name.split('.')[0]).with_suffix('')
print(f"Arquivos extraídos em: {data_dir}")


Após o download, você deverá ter uma cópia do conjunto de dados disponível. Há um total de 3.670 imagens:

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

Aqui estão algumas rosas:

In [ ]:
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
PIL.Image.open(str(roses[1]))

E algumas tulipas:

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
PIL.Image.open(str(tulips[0]))

In [ ]:
PIL.Image.open(str(tulips[1]))

## Load data using a Keras utility

Next, load these images off disk using the helpful `tf.keras.utils.image_dataset_from_directory` utility. This will take you from a directory of images on disk to a `tf.data.Dataset` in just a couple lines of code. If you like, you can also write your own data loading code from scratch by visiting the [Load and preprocess images](../load_data/images.ipynb) tutorial.

## Carregar dados usando um utilitário do Keras

Em seguida, carregue essas imagens usando o utilitário `tf.keras.utils.image_dataset_from_directory`.

Isso o levará de um diretório de imagens no disco para um `tf.data.Dataset` em apenas algumas linhas de código.

### Crie o conjunto de dados(_dataset_)

Defina alguns parâmetros para o carregador:

In [48]:
batch_size = 32
img_height = 180
img_width = 180

É uma boa prática usar uma divisão de validação ao desenvolver seu modelo. Use 80% das imagens para treinamento e 20% para validação.

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Você pode encontrar os nomes das classes no atributo `class_names` nesses conjuntos de dados. Eles correspondem aos nomes dos diretórios em ordem alfabética.

In [ ]:
class_names = train_ds.class_names
print(class_names)

## Visualize os dados

Aqui estão as primeiras nove imagens do conjunto de dados de treinamento:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Você passará esses conjuntos de dados para o método `Model.fit` do Keras para treinamento mais adiante neste tutorial.

Se desejar, você também pode iterar manualmente o conjunto de dados e recuperar lotes de imagens:

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

O `image_batch` é um tensor com a forma `(32, 180, 180, 3)`.
- Trata-se de um lote de 32 imagens de formato `180x180x3` (a última dimensão refere-se aos canais de cores RGB).
- O `label_batch` é um tensor da forma `(32,)`, esses são os rótulos correspondentes às 32 imagens.

Você pode chamar `.numpy()` nos tensores `image_batch` e `labels_batch` para convertê-los em um `numpy.ndarray`.

## Configure o conjunto de dados para desempenho

Certifique-se de usar o pré-busca com buffer(_buffered prefetching_), para que você possa obter dados do disco sem que a E/S se torne bloqueada. Esses são dois métodos importantes que você deve usar ao carregar dados:

- O `Dataset.cache` mantém as imagens na memória depois que elas são carregadas do disco durante a primeira época. Isso garantirá que o conjunto de dados não se torne um gargalo durante o treinamento do modelo. Se o conjunto de dados for muito grande para caber na memória, você também poderá usar esse método para criar um cache de alto desempenho no disco.
- O `Dataset.prefetch` sobrepõe o pré-processamento de dados e a execução do modelo durante o treinamento.

Se estiver interessado, pode saber mais sobre os dois métodos, bem como sobre como armazenar dados em cache no disco, na seção *Prefetching* do guia [Better performance with the tf.data API](https://www.tensorflow.org/guide/data_performance?hl=pt-br).

In [54]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Padronize os dados

Os valores do canal RGB estão no intervalo `[0, 255]`. Isso não é ideal para uma rede neural; em geral, você deve procurar tornar os valores de entrada pequenos.

Aqui, você padronizará os valores para que fiquem no intervalo `[0, 1]` usando `tf.keras.layers.Rescaling`:

In [55]:
normalization_layer = layers.Rescaling(1./255)

Há duas maneiras de usar essa camada. Você pode aplicá-la ao conjunto de dados chamando `Dataset.map`:

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Observe que os valores de pixel estão agora em `[0,1]`.
print(np.min(first_image), np.max(first_image))

Or, you can include the layer inside your model definition, which can simplify deployment. Use the second approach here.

Note: You previously resized images using the `image_size` argument of `tf.keras.utils.image_dataset_from_directory`. If you want to include the resizing logic in your model as well, you can use the `tf.keras.layers.Resizing` layer.

## Um modelo básico do Keras

### Criar o modelo

O modelo Keras [Sequential](https://www.tensorflow.org/guide/keras/sequential_model) consiste em três blocos de convolução (`tf.keras.layers.Conv2D`) com uma camada de pooling máximo (`tf.keras.layers.MaxPooling2D`) em cada um deles.

Há uma camada totalmente conectada (`tf.keras.layers.Dense`) com 128 unidades na parte superior, que é ativada por uma função de ativação ReLU (`'relu'`).

Esse modelo não foi ajustado para alta acurácia; o objetivo deste tutorial é mostrar uma abordagem padrão.

In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

### Compilar o modelo

Para este tutorial, escolha o otimizador `tf.keras.optimizers.Adam` e a função de perda `tf.keras.losses.SparseCategoricalCrossentropy`.

Para visualizar a acurácia do treinamento e da validação para cada época de treinamento, passe o argumento `metrics` para `Model.compile`.

In [58]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Resumo do modelo

Visualize todas as camadas da rede usando o método `Model.summary` do Keras:

In [ ]:
model.summary()

### Treine o modelo

Treine o modelo para 10 épocas com o método `Model.fit` do Keras:

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## Visualize os resultados do treinamento

Crie gráficos da perda e da acurácia nos conjuntos de treinamento e validação:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Acurácia de Treino')
plt.plot(epochs_range, val_acc, label='Acurácia de Validação')
plt.legend(loc='lower right')
plt.title('Acurácia de Treino e Validação')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Perda de Treino')
plt.plot(epochs_range, val_loss, label='Perda de Validação')
plt.legend(loc='upper right')
plt.title('Perdas de Treino e Validação')
plt.show()

Os gráficos mostram que a acurácia do treinamento e a acurácia da validação estão fora de uma grande margem, e o modelo alcançou apenas cerca de 60% de acurácia no conjunto de validação.

As seções do tutorial a seguir mostram como inspecionar o que deu errado e tentar aumentar o desempenho geral do modelo.

## Sobreajuste(_Overfitting_)

Nos gráficos acima, a acurácia do treinamento está aumentando linearmente ao longo do tempo, ao passo que a acurácia da validação fica parada em torno de 60% no processo de treinamento. Além disso, a diferença entre a acurácia do treinamento e da validação é perceptível - um sinal de [overfitting](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit).

Quando há um pequeno número de exemplos de treinamento, o modelo às vezes aprende com ruídos ou detalhes indesejados dos exemplos de treinamento - a ponto de afetar negativamente o desempenho do modelo em novos exemplos. Esse fenômeno é conhecido como sobreajuste. Isso significa que o modelo terá dificuldade para generalizar em um novo conjunto de dados.

Há várias maneiras de combater o ajuste excessivo no processo de treinamento. Neste tutorial, você usará *data augmentation* e adicionará *dropout* ao seu modelo.

## Aumento de Dados(_Data augmentation_)

O overfitting geralmente ocorre quando há um pequeno número de exemplos de treinamento.

O [Data augmentation](https://www.tensorflow.org/tutorials/images/data_augmentation?hl=pt-br) adota a abordagem de gerar dados de treinamento adicionais a partir dos exemplos existentes, aumentando-os com transformações aleatórias que geram imagens de aparência crível. Isso ajuda a expor o modelo a mais aspectos dos dados e a generalizar melhor.

Você implementará o aumento de dados usando as seguintes camadas de pré-processamento do Keras: `tf.keras.layers.RandomFlip`, `tf.keras.layers.RandomRotation` e `tf.keras.layers.RandomZoom`. Elas podem ser incluídas em seu modelo como outras camadas e executadas na GPU.

In [62]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

Visualize alguns exemplos aumentados aplicando o aumento de dados à mesma imagem várias vezes:

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

Na próxima etapa, você adicionará o aumento de dados ao seu modelo antes do treinamento.

## Dropout

Outra técnica para reduzir o overfitting é introduzir a regularização [dropout](https://developers.google.com/machine-learning/glossary#dropout_regularization){:.external} na rede.

Quando você aplica o dropout a uma camada, ele elimina aleatoriamente (definindo a ativação como zero) um número de unidades de saída da camada durante o processo de treinamento.
- O dropout usa um número fracionário como seu valor de entrada, na forma de 0,1, 0,2, 0,4 etc.
- Isso significa eliminar 10%, 20% ou 40% das unidades de saída aleatoriamente da camada aplicada.

Crie uma nova rede neural com `tf.keras.layers.Dropout` antes de treiná-la usando as imagens aumentadas:

In [66]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

## Compile e treine o modelo

In [67]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## Visualize os resultados do treinamento

Depois de aplicar o aumento de dados e `tf.keras.layers.Dropout`, há menos overfitting do que antes, e a acurácia do treinamento e da validação estão mais alinhadas:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Acurácia de Treino')
plt.plot(epochs_range, val_acc, label='Acurácia de Validação')
plt.legend(loc='lower right')
plt.title('Acurácia de Treino e Validação')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Perda de Treino')
plt.plot(epochs_range, val_loss, label='Perda de Validação')
plt.legend(loc='upper right')
plt.title('Perdas de Treino e Validação')
plt.show()

## Prever com base em novos dados

Use seu modelo para classificar uma imagem que não foi incluída nos conjuntos de treinamento ou validação.

Observação: As camadas de aumento de dados e de abandono estão inativas no momento da inferência.

In [ ]:
sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = tf.keras.utils.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "Essa imagem provavelmente é uma {} com {:.2f} por cento de confiança."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

# Exibe a imagem
# Remove a dimensão do batch para visualização
img_array = np.squeeze(img_array, axis=0)

# Plota a imagem
plt.imshow(img_array.astype('uint8'))
plt.title(
    "Essa imagem provavelmente é uma {} com {:.2f} por cento de confiança."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)
plt.axis('off')  # Remove os eixos para melhor visualização
plt.show()